In [1]:
import xgboost as xgb
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from datetime import datetime

from model_utils import PreProcess
from constants import Y_LABEL, X_LABELS

## Get Train and test data 

In [2]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
train["is_training"] = True
test["is_training"] = False
train_test = pd.concat([train, test])


In [3]:
p = PreProcess(data=train_test, label_encoder_cols=["country", "store", "product"])
train, test = p.train_test


# xgb

In [4]:
X_train, X_val, y_train, y_val = train_test_split(
    train[X_LABELS], train[Y_LABEL], test_size=0.2, random_state=42
)


In [5]:
model = xgb.XGBRegressor(n_estimators=1000)
model.fit(X_train, y_train)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [6]:
y_pred = model.predict(X_val)
print("the r2 is: ", round(metrics.r2_score(y_pred=y_pred, y_true=y_val), 2))
print("the mse is: ", round(metrics.mean_squared_error(y_pred=y_pred, y_true=y_val), 2))


the r2 is:  0.97
the mse is:  543.26


# Test predictions

In [7]:
test_preds = model.predict(test[X_LABELS])


In [8]:
sub = pd.read_csv('../data/sample_submission.csv')
sub["num_sold"] = test_preds
sub.to_csv(f"../data/submission_{datetime.now()}.csv", index = False)
